In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch

from Generation.models import *
from transformers import (
    GPT2Config,
    GPT2Tokenizer
)
from transformers import AutoConfig

In [246]:
cache_dir='../HULK_new/Saved_models/'


#gen_model_name_or_path='../HULK_new/Counterspeech/Saved_Models/Generator/Create_debate_DialoGPT-medium/'
#gen_model_name_or_path='../HULK_new/Counterspeech/Saved_Models/Generator/CONAN_DialoGPT-medium/'
gen_model_name_or_path='microsoft/DialoGPT-medium'

tokenizer = GPT2Tokenizer.from_pretrained(gen_model_name_or_path, cache_dir=cache_dir)
model = Model_Generation.from_pretrained(gen_model_name_or_path,cache_dir=cache_dir)
#model = model.to(device)
model = model.float()

gedi_model_name_or_path = '../HULK_new/Counterspeech/Saved_Models/Discriminator/Toxicity_gedi_gpt2_toxic/'
gedi_model = Model_Generation.from_pretrained(gedi_model_name_or_path,cache_dir=cache_dir)
#gedi_model.to(device)

no logit scale or bias initialized for gpt2


Some weights of Model_Generation were not initialized from the model checkpoint at microsoft/DialoGPT-medium and are newly initialized: ['transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.12.attn.masked_bias', 'transformer.h.13.attn.masked_bias', 'transformer.h.14.attn.masked_bias', 'transformer.h.15.attn.masked_bias', 'transformer.h.16.attn.masked_bias', 'transformer.h.17.attn.masked_bias', 'transformer.h.18.attn.masked_bias', 'transformer.h.19.attn.masked_bias', 'transformer.h.20.attn.masked_bias', 'transformer.h.21.attn.masked_bias', 'transformer.h.22.attn.masked_bias', 'transformer.h.23.attn

Loaded config and bias


In [247]:
params = {
    'sep_token':'<|endoftext|>',
    'max_generation_length': 80,
    'min_generation_length':40,
    'max_input_length':128,
    'num_beams':1,
    'no_repeat_ngram_size':5,
    'repitition_penalty': 1.5,
    'control_type':'gedi',
    'k':100,
    'p':0.92,
    'filter_p':0.9,
    'target_p':0.9,
    'disc_weight':30,
    'class_bias':0,
    'sample':True,
    'temperature':1.0,
    'early_stopping':True,
    'model_path':'gpt2-medium',
    'dataset_hate':'CONAN',
    'task_name':[('Emotion','joy')],
    'coefficient':[4.5],
    'save_path': './../HULK_new/Counterspeech/Results/',
    'device': 'cuda',
    'batch_size':4,
    'cache_path':'./../HULK_new/Saved_models/',
    'generation_method':'huggingface',
    'class_bias':0,
}

In [251]:
prompt = "I hate all niggers <|endoftext|>"

start_len=0
text_ids = tokenizer.encode(prompt)
encoded_prompts=torch.LongTensor(text_ids).unsqueeze(0)
# multi_code = tokenizer.encode(secondary_code)
beam_outputs = model.generate(
            controller_alphas=None,
            controller_list=[gedi_model],
            control_type='gedi',
            positive_class='true',
            negative_class='false',
            tokenizer=tokenizer,
            class_bias=params['class_bias'],
            filter_p=params['filter_p'],
            target_p=params['target_p'],
            disc_weight=params['disc_weight'],
            input_ids=encoded_prompts, 
            pad_token_id         = tokenizer.eos_token_id,
            max_length           = params['max_generation_length']+len(encoded_prompts[0]),
            min_length           = params['min_generation_length']+len(encoded_prompts[0]),
            top_k                = params["k"],
            top_p                = params["p"],
            repetition_penalty   = params["repitition_penalty"],
            temperature          = params["temperature"],
            num_beams            = params['num_beams'], 
            do_sample            = params['sample'],
            no_repeat_ngram_size = params['no_repeat_ngram_size'],  
            early_stopping       = params['early_stopping']
        )
reply = (tokenizer.decode(beam_outputs[0])).split(params['sep_token'])[1]
print("Prompt:",prompt)
print("Reply:",reply)


inside own function
torch.Size([1, 7])
tensor([40])
tensor([40])
tensor([466])
tensor([466])
tensor([428])
tensor([428])
tensor([326])
tensor([326])
tensor([345])
tensor([[7942, 5195,  318,  340,  326]])
torch.Size([2, 5])
torch.Size([2, 50257])
torch.Size([1, 50257])
tensor([790])
tensor([996])
torch.Size([2, 50257])
torch.Size([1, 50257])
tensor([996])
tensor([345])
torch.Size([2, 50257])
torch.Size([1, 50257])
tensor([314])
tensor([766])
torch.Size([2, 50257])
torch.Size([1, 50257])
tensor([766])
tensor([27257])
torch.Size([2, 50257])
torch.Size([1, 50257])
tensor([27257])
tensor([345])
torch.Size([2, 50257])
torch.Size([1, 50257])
tensor([257])
tensor([9811])
torch.Size([2, 50257])
torch.Size([1, 50257])
tensor([2912])
tensor([837])
torch.Size([2, 50257])
torch.Size([1, 50257])
tensor([837])
tensor([837])
torch.Size([2, 50257])
torch.Size([1, 50257])
tensor([837])
tensor([428])
torch.Size([2, 50257])
torch.Size([1, 50257])
tensor([428])
tensor([11783])
torch.Size([2, 50257])
torch.

In [222]:
tokenizer.eos_token_id

50256

In [14]:
encoded_prompts

tensor([[ 1639,   389,   257,  1928,  2475,   220, 50256]])